In [1]:
import os,sys
pwd = os.getcwd()
parent = '/'.join(pwd.split('/')[:-1])
sys.path.insert(0,parent)
#os.chdir(parent)
#%%
print('-'*30)
print(os.getcwd())
print('-'*30)
#%%
import torch
import torch.optim as optim
import torch.nn as nn
import pandas as pd
from DAZLE import DAZLE
from AWA2DataLoader import AWA2DataLoader
from helper_func import eval_zs_gzsl,visualize_attention#,get_attribute_attention_stats
import importlib
import numpy as np
import matplotlib.pyplot as plt

------------------------------
C:\Users\Sushree.Behera\Sushree\GZSL_Implementation
------------------------------


In [2]:
idx_GPU = 0
device = torch.device("cuda:{}".format(idx_GPU) if torch.cuda.is_available() else "cpu")
NFS_path = 'C:/Sushree/Jio_Institute/Dataset/'
dataloader = AWA2DataLoader(NFS_path,device)


C:/Sushree/Jio_Institute/Dataset/
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
AWA2
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Balance dataloader
_____
C:/Sushree/Jio_Institute/Dataset/data/AWA2/feature_map_ResNet_101_AWA2.hdf5
Expert Attr
threshold at zero attribute with negative value


In [3]:
def get_lr(optimizer):
    lr = []
    for param_group in optimizer.param_groups:
        lr.append(param_group['lr'])
    return lr

In [4]:
seed = 214#214
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

batch_size = 50
nepoches = 20
niters = dataloader.ntrain * nepoches//batch_size
dim_f = 2048
dim_v = 300
init_w2v_att = dataloader.w2v_att
att = dataloader.att #dataloader.normalize_att#
att[att<0] = 0
normalize_att = dataloader.normalize_att
#assert (att.min().item() == 0 and att.max().item() == 1)

trainable_w2v = True
lambda_ = 0.1#0.1
bias = 0
prob_prune = 0
uniform_att_1 = False
uniform_att_2 = False

seenclass = dataloader.seenclasses
unseenclass = dataloader.unseenclasses
desired_mass = 1#unseenclass.size(0)/(seenclass.size(0)+unseenclass.size(0))
report_interval = niters//nepoches#10000//batch_size#

model = DAZLE(dim_f,dim_v,init_w2v_att,att,normalize_att,
            seenclass,unseenclass,
            lambda_,
            trainable_w2v,normalize_V=True,normalize_F=True,is_conservative=True,
            uniform_att_1=uniform_att_1,uniform_att_2=uniform_att_2,
            prob_prune=prob_prune,desired_mass=desired_mass, is_conv=False,
            is_bias=True)
model.to(device)

setup = {'pmp':{'init_lambda':0.1,'final_lambda':0.1,'phase':0.8},
         'desired_mass':{'init_lambda':-1,'final_lambda':-1,'phase':0.8}}
print(setup)

params_to_update = []
params_names = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        params_names.append(name)
        print("\t",name)
#%%
lr = 0.0001
weight_decay = 0.0001#0.000#0.#
momentum = 0.#0.#
#%%
lr_seperator = 1
lr_factor = 1
print('default lr {} {}x lr {}'.format(params_names[:lr_seperator],lr_factor,params_names[lr_seperator:]))
optimizer  = optim.RMSprop( params_to_update ,lr=lr,weight_decay=weight_decay, momentum=momentum)
print('-'*30)
print('learing rate {}'.format(lr))
print('trainable V {}'.format(trainable_w2v))
print('lambda_ {}'.format(lambda_))
print('optimized seen only')
print('optimizer: RMSProp with momentum = {} and weight_decay = {}'.format(momentum,weight_decay))
print('-'*30)

------------------------------
Configuration
loss_type CE
normalize V
normalize F
training to exclude unseen class [seen upperbound]
Init word2vec
Linear model
loss_att BCEWithLogitsLoss()
Bilinear attention module
******************************
Measure w2v deviation
Compute Pruning loss Parameter containing:
tensor(0)
Add one smoothing
Second layer attenion conditioned on image features
------------------------------
No sigmoid on attr score
{'pmp': {'init_lambda': 0.1, 'final_lambda': 0.1, 'phase': 0.8}, 'desired_mass': {'init_lambda': -1, 'final_lambda': -1, 'phase': 0.8}}
	 V
	 W_1
	 W_2
	 W_3
default lr ['V'] 1x lr ['W_1', 'W_2', 'W_3']
------------------------------
learing rate 0.0001
trainable V True
lambda_ 0.1
optimized seen only
optimizer: RMSProp with momentum = 0.0 and weight_decay = 0.0001
------------------------------


C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.init_w2v_att = F.normalize(torch.tensor(init_w2v_att))
C:\Users\Sushree.Behera\Sushree\GZSL_Implementation\DAZLE.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.att = nn.Parameter(F.normalize(torch.tensor(att)),requires_grad = False)


In [5]:
best_performance = [0,0,0,0]
for i in range(0,niters):
    model.train()
    optimizer.zero_grad()
    
    batch_label, batch_feature, batch_att = dataloader.next_batch(batch_size)
    out_package = model(batch_feature)
    
    in_package = out_package
    in_package['batch_label'] = batch_label
    
    out_package=model.compute_loss(in_package)
    loss,loss_CE,loss_cal = out_package['loss'],out_package['loss_CE'],out_package['loss_cal']
    
    loss.backward()
    optimizer.step()
    if i%report_interval==0:
        print('-'*30)
        acc_seen, acc_novel, H, acc_zs = eval_zs_gzsl(dataloader,model,device,bias_seen=-bias,bias_unseen=bias)
        
        if H > best_performance[2]:
            best_performance = [acc_seen, acc_novel, H, acc_zs]
        stats_package = {'iter':i, 'loss':loss.item(), 'loss_CE':loss_CE.item(),
                         'loss_cal': loss_cal.item(),
                         'acc_seen':best_performance[0], 'acc_novel':best_performance[1], 'H':best_performance[2], 'acc_zs':best_performance[3]}
        
        print(stats_package)

------------------------------
bias_seen 0 bias_unseen 0
{'iter': 0, 'loss': 3.956040143966675, 'loss_CE': 3.9095566272735596, 'loss_cal': 0.4648360013961792, 'acc_seen': 0, 'acc_novel': 0, 'H': 0, 'acc_zs': 0}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 470, 'loss': 1.3515682220458984, 'loss_CE': 1.2346607446670532, 'loss_cal': 1.1690747737884521, 'acc_seen': 0.5439391732215881, 'acc_novel': 0.605766773223877, 'H': 0.5731905254751752, 'acc_zs': 0.6462183594703674}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 940, 'loss': 0.9183818697929382, 'loss_CE': 0.7963429093360901, 'loss_cal': 1.2203896045684814, 'acc_seen': 0.7085906267166138, 'acc_novel': 0.6012459993362427, 'H': 0.6505197228518068, 'acc_zs': 0.6683871150016785}
------------------------------
bias_seen 0 bias_unseen 0
{'iter': 1410, 'loss': 0.7530431151390076, 'loss_CE': 0.6216723322868347, 'loss_cal': 1.3137075901031494, 'acc_seen': 0.7400235533714294, 'acc_novel': 0.5982651710510254